**Preprocessing script for images**

NB: The first script is run locally on my own computer, before uploading uniquely named images to google colaboratory.

The images are first uploaded to google colaboratory. 

In [1]:
# load necessary modules
import glob
import re
import os
from google.colab import files

# define path
path = "/content/"
os.chdir(path)
    
# create folder for all images to be put in
if not os.path.exists('data'):
    os.makedirs('data')

# define new path where we will put the zip file with all images
new_path = "/content/data/"
os.chdir(new_path)
print(new_path)
    
# import zipfile 'transmission_VY53sLXHyJ6H_0G5z021ZWZdA.zip' with all data into the data folder
uploaded = files.upload()

# also import 'Drawing_ID.csv'
uploaded = files.upload()

# unzip the zipfile
!unzip 'transmission_VY53sLXHyJ6H_0G5z021ZWZdA.zip' #"my_archive.zip"

/content/data/


Saving transmission_VY53sLXHyJ6H_0G5z021ZWZdA.zip to transmission_VY53sLXHyJ6H_0G5z021ZWZdA.zip


Streaming output truncated to the last 5000 lines.
  inflating: 1/2/cond_1_stim_2.json  
  inflating: 1/2/cond_1_stim_2.png   
  inflating: 1/2/cond_1_stim_3.json  
  inflating: 1/2/cond_1_stim_3.png   
  inflating: 1/2/cond_2_stim_1.json  
  inflating: 1/2/cond_2_stim_1.png   
  inflating: 1/2/cond_2_stim_2.json  
  inflating: 1/2/cond_2_stim_2.png   
  inflating: 1/2/cond_2_stim_3.json  
  inflating: 1/2/cond_2_stim_3.png   
  inflating: 1/2/cond_3_stim_1.json  
  inflating: 1/2/cond_3_stim_1.png   
  inflating: 1/2/cond_3_stim_2.json  
  inflating: 1/2/cond_3_stim_2.png   
  inflating: 1/2/cond_3_stim_3.json  
  inflating: 1/2/cond_3_stim_3.png   
  inflating: 1/2/cond_4_stim_1.json  
  inflating: 1/2/cond_4_stim_1.png   
  inflating: 1/2/cond_4_stim_2.json  
  inflating: 1/2/cond_4_stim_2.png   
  inflating: 1/2/cond_4_stim_3.json  
  inflating: 1/2/cond_4_stim_3.png   
  inflating: 1/2/session.json        
  inflating: 1/2/tutorial1.json      
  inflating: 1/2/tutorial1.png       

Now, assign unique names to images using the csv-file created in the R preprocessing script.

In [2]:
"""
Created on Thu Nov 12, 2020

@author: Sophia Marthine Kleist Karlson
"""
## Script that renames drawings to unique names and puts them all in the same folder


# import the necessary packages
import pandas as pd
import cv2
import os

# set path
path = '/content/' #"C:/Users/Sophia/Documents/Social Transmission Study/Analysis of drawings/data/"
os.chdir(path)

# create folder for all images to be put in
if not os.path.exists('all_drawings'):
    os.makedirs('all_drawings')

# Import the csv file with image paths and unique names
Drawing_IDs = pd.read_csv('data/Drawing_IDs.csv')

# make the image path column to a list
image_path_list = Drawing_IDs[["image_path"]]
image_path_list = image_path_list["image_path"].tolist()

# make the image ID column to a list
image_ID_list = Drawing_IDs[["Drawing_ID"]]
image_ID_list = image_ID_list["Drawing_ID"].tolist()

# Loop that reads the image paths and saves them into the all_drawings folder with their unique names
for i in range(len(image_ID_list)):
    drawing = cv2.imread(image_path_list[i])
    drawing_ID = "all_drawings/" + image_ID_list[i] + ".png"
    cv2.imwrite(drawing_ID, drawing)


All drawings are now resized to 400x400 px.

In [3]:
"""
Created on Fri Nov 13, 2020

@author: Sophia Marthine Kleist Karlson
"""
## Script that resizes drawings to be 400x400px and puts them all in the same folder


# load necessary modules
import cv2
import glob
import re
import os


# define path where all drawings are
path = "/content/"#"C:/Users/Sophia/Documents/Social Transmission Study/Analysis of drawings/data/all_drawings/"
os.chdir(path)
print(path)

# make a list of all the drawings
img_list = glob.glob(path + "all_drawings/*.png")
img_list.sort() # sort the list alphabetically

print(len(img_list))

# make empty list to be filled with filenames
id = []

# make folder for the resized images
if not os.path.exists('resized'):#'all_resized_drawings'
    os.makedirs('resized')

# loop that resizes each image to 400x400 px
for i in range(len(img_list)):
  id.append(re.findall('/content/all_drawings/(Chain_\d+_Gen_\d+_Cond_\d+_Source_\d+).png', img_list[i])[0])
  src = cv2.imread(img_list[i], cv2.IMREAD_UNCHANGED)

  # set a new height and width in pixels
  new_height = 400
  new_width = 400

  # dsize
  dsize = (new_width, new_height)

  # resize image
  output = cv2.resize(src, dsize)
  
  # write the resized image and put it in the all_resized_drawings folder
  cv2.imwrite('/content/resized/{}.png'.format(id[i]), output) #'C:/Users/Sophia/Documents/Social Transmission Study/Analysis of drawings/data/all_drawings/all_resized_drawings/{}.png'



/content/
1680


The resized images are blurred using cv.blur(), using the **averaging** function, and then converted to jpeg2000 format, the file sizes of which are put into a dataframe and saved as a csv. This csv as well as the resized images, blurred images and jpeg2000 files are zipped and downloaded.

In [4]:
# import the necessary packages
import PIL
from PIL import Image
from pathlib import Path
import numpy as np
import pandas as pd
import glob
import os
import re
import cv2
import cv2 as cv
from google.colab import files
from zipfile import ZipFile

# Define function
def complexity_comparison_function(img_list):
  
  # make directory to put jp2 files
  if not os.path.exists('jpeg_compressed_files_comparison'):
    os.makedirs('jpeg_compressed_files_comparison')

  # make directory to put blurred png files
  if not os.path.exists('blurred_images'):
    os.makedirs('blurred_images')
    
  # prepare dataframe
  columns = ['Drawing_ID', 'Complexity_original', 'Complexity_convolution']
  index = np.arange(0)
  Complexity_comparison_data = pd.DataFrame(columns=columns, index = index)

  # list for id's
  id = []
  
  # convert png to jp2
  for i in range(len(img_list)):
    id.append(re.findall('/content/resized/(Chain_\d+_Gen_\d+_Cond_\d+_Source_\d+).png', img_list[i])[0]) 
    
    #complexity of original:
    img_original = Image.open(img_list[i])
    img_original.convert("RGBA").save("jpeg_compressed_files_comparison/Original_{}.jp2".format(id[i]), 'JPEG2000', quality_mode='dB', quality_layers=[80])
    
    # read image for blurring
    img_blur = cv.imread(img_list[i])

    # convolution
    kernel = np.ones((5,5),np.float32)/25
    dst = cv.filter2D(img_blur,-1,kernel)
    cv2.imwrite('/content/blurred_images/Convolution_{}.png'.format(id[i]), dst)

    # jpeg2000 convertion of blurred images
    img_convolution = Image.open("/content/blurred_images/Convolution_{}.png".format(id[i]))
    img_convolution.convert("RGBA").save("jpeg_compressed_files_comparison/Convolution_{}.jp2".format(id[i]), 'JPEG2000', quality_mode='dB', quality_layers=[80])

  # list of original jp2 files
  img_jp2_original = glob.glob("jpeg_compressed_files_comparison/Original_*.jp2")
  img_jp2_original.sort() #important! Else the IDs and complexity scores won't match

  # list of convolution jp2 files
  img_jp2_convolution = glob.glob("jpeg_compressed_files_comparison/Convolution_*.jp2")
  img_jp2_convolution.sort()

  # save size and ID to dataframe
  for i in range(len(img_jp2_original)): 
    size_original = (Path(img_jp2_original[i]).stat().st_size)
    size_convolution = (Path(img_jp2_convolution[i]).stat().st_size)

    Complexity_comparison_data = Complexity_comparison_data.append({
      'Drawing_ID': id[i],
      'Complexity_original': size_original, 
      'Complexity_convolution': size_convolution
    }, ignore_index=True)

  return Complexity_comparison_data


# define path where all drawings are
path = "/content/resized/" 
images = glob.glob(path + "*.png")
print(len(images))
images.sort() #important! Else the IDs and complexity scores won't match

# run function on all drawings
complexity_comparison_data = complexity_comparison_function(images)

# print complexity scores and ID's
print(complexity_comparison_data)

# prepare logfilename and save
logfilename = "complexity_comparison.csv"
complexity_comparison_data.to_csv(logfilename)

# prepare zipfile
zipObj = ZipFile('image_processing.zip', 'w')

# add complexity_comparison.csv to the zipfile
zipObj.write('complexity_comparison.csv')

# prepare lists of resized and blurred images and jp2 files
resized = glob.glob('resized/' + "*.png")
blurred = glob.glob('blurred_images/' + "*.png")
jpeg2000 = glob.glob('jpeg_compressed_files_comparison/' + "*.jp2")

# loop over the resized and blurred images and add them to the zipfile
for i in range(len(blurred)):
  # Add multiple files to the zipfile
  zipObj.write(resized[i])
  zipObj.write(blurred[i])

# loop over the jp2 files and add them to the zipfile
for i in range(len(jpeg2000)):
  zipObj.write(jpeg2000[i])

# close the zipfile
zipObj.close()

# download zipfile
files.download("image_processing.zip") 

1680
                         Drawing_ID Complexity_original Complexity_convolution
0     Chain_0_Gen_0_Cond_1_Source_1               35634                  28713
1     Chain_0_Gen_0_Cond_1_Source_5               35430                  28887
2     Chain_0_Gen_0_Cond_1_Source_7               37926                  30597
3     Chain_0_Gen_0_Cond_2_Source_3               40218                  32625
4     Chain_0_Gen_0_Cond_2_Source_6               37623                  30342
...                             ...                 ...                    ...
1675  Chain_9_Gen_6_Cond_3_Source_2               17451                  14583
1676  Chain_9_Gen_6_Cond_3_Source_8               29100                  23664
1677  Chain_9_Gen_6_Cond_4_Source_3               32382                  26709
1678  Chain_9_Gen_6_Cond_4_Source_5               23694                  19647
1679  Chain_9_Gen_6_Cond_4_Source_6               31458                  26199

[1680 rows x 3 columns]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [47]:
print(len(jpeg2000))

3360
